In [21]:
from __future__ import (absolute_import, division, print_function, unicode_literals)

In [22]:
%matplotlib
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [20, 10]
%reload_ext autoreload
%autoreload 2

Using matplotlib backend: MacOSX


In [23]:
import datetime # For datetime objects
import os.path # To manage paths
import sys # To find out the script name (in argv[0])
import pandas as pd
import quandl

In [24]:
import backtrader as bt
import backtrader.indicators as btind
import backtrader.feeds as btfeeds

In [406]:
class PandasData(btfeeds.DataBase):
    '''
    The ``dataname`` parameter inherited from ``feed.DataBase`` is the pandas
    DataFrame
    '''

    params = (
        # Possible values for datetime (must always be present)
        #  None : datetime is the "index" in the Pandas Dataframe
        #  -1 : autodetect position or case-wise equal name
        #  >= 0 : numeric index to the colum in the pandas dataframe
        #  string : column name (as index) in the pandas dataframe
        ('datetime', None),

        # Possible values below:
        #  None : column not present
        #  -1 : autodetect position or case-wise equal name
        #  >= 0 : numeric index to the colum in the pandas dataframe
        #  string : column name (as index) in the pandas dataframe
        ('open', 'Open'),
        ('high', 'High'),
        ('low', 'Low'),
        ('close', 'Close'),
        ('volume', 'Volume'),
        ('openinterest', None),
    )

In [403]:
# Create a Stratey
class TestStrategyB(bt.Strategy):
    params = (
        ('mafast', 1),
        ('maslow', 15),
        ('printlog', False),
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function for this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Add a MovingAverageSimple indicator
        self.sma_slow = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maslow)
        
        self.sma_fast = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.mafast)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' % 
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            elif order.issell(): # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm: %.2f' % 
                     (order.executed.price,
                      order.executed.value,
                      order.executed.comm))

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.sma_fast[0] > self.sma_slow[0]:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if self.sma_fast[0] < self.sma_slow[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

    def stop(self):
        self.log('(MA Fast %2d Slow %2d) Ending Value %.2f' %
                 (self.params.mafast, self.params.maslow, self.broker.getvalue()), doprint=True)

In [389]:
# Create a Stratey
class TestStrategyA(bt.Strategy):
    params = (
        ('mafast', 1),
        ('maslow', 15),
        ('printlog', False),
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function for this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Add a MovingAverageSimple indicator
        self.sma_slow = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maslow)
        
        self.sma_fast = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.mafast)
        
        self.crossover = bt.indicators.CrossOver(self.sma_fast, self.sma_slow)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' % 
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            elif order.issell(): # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm: %.2f' % 
                     (order.executed.price,
                      order.executed.value,
                      order.executed.comm))

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            if self.crossover > 0:
            # Not yet ... we MIGHT BUY if ...
            #if self.sma_fast[0] > self.sma_slow[0]:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if self.crossover < 0:
            #if self.sma_fast[0] < self.sma_slow[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

    def stop(self):
        self.log('(MA Fast %2d Slow %2d) Ending Value %.2f' %
                 (self.params.mafast, self.params.maslow, self.broker.getvalue()), doprint=True)


In [418]:
# Create a cerebro entity
cerebro = bt.Cerebro()

In [419]:
cerebro.addstrategy(TestStrategyB, mafast = 1, maslow = 7, printlog = True)

0

In [260]:
# Optimize thestrategy
strats = cerebro.optstrategy(
    TestStrategy,
    mafast=1,
    maslow=range(10, 100))

In [220]:
# Datas are in a subfolder of the samples. Need to find where the script is
# because it could have been called from anywhere
PATH = 'datas'
#modpath = os.path.dirname(os.path.abspath(sys.argv[0])); modpath
datapath = PATH + '/orcl-1995-2014.txt'; datapath

'datas/orcl-1995-2014.txt'

In [420]:
# Get ETH prices for GDAX exchange
eth = quandl.get("GDAX/ETH_BTC") 
eth.columns = [u'Open', u'High', u'Low', u'Volume'] # rename columns
eth['Close'] = eth['Open'].shift(-1)
eth = eth[['Open','High','Low','Close','Volume']]
eth = eth.dropna()
eth.tail()

,Open,High,Low,Close,Volume
Date,,,,,
2018-06-28,0.07137,0.07217,0.07072,0.07178,4696.899976
2018-06-29,0.07178,0.07199,0.06915,0.06957,7667.003716
2018-06-30,0.06957,0.07175,0.06957,0.07050,3609.827617
2018-07-01,0.07050,0.07140,0.07050,0.07097,2738.517233
2018-07-02,0.07097,0.07239,0.07056,0.07215,7368.196072


In [421]:
# Use same time periods and dates as medium article
training_cutoff_date = "2017-05-19" 
training_cutoff_i = eth.index.get_loc(training_cutoff_date)
eth = eth.loc[:'2018-05-13'] # data up to 2018-05-13

In [337]:
train = eth.iloc[:training_cutoff_i]

In [321]:
test = eth.iloc[training_cutoff_i:]

In [422]:
full = eth

In [424]:
data = bt.feeds.PandasData(dataname=full)

In [423]:
# Create a Data Feed
data = bt.feeds.YahooFinanceCSVData(
    dataname=datapath,
    # Do not pass values before this date
    fromdate=datetime.datetime(2000, 1, 1),
    # Do not pass values before this date
    todate=datetime.datetime(2000, 12, 31),
    # Do not pass values after this date
    reverse=False)

In [425]:
# Add the Data Feed to Cerebro
cerebro.adddata(data)

In [426]:
# Set our desired cash start
cerebro.broker.setcash(1000.0)

In [427]:
# Add a FixedSize sizer according to the stake
cerebro.addsizer(bt.sizers.PercentSizer, percents=99.75)

In [428]:
# Set the commission
cerebro.broker.setcommission(commission=0.0025)

In [429]:
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 1000.00


In [430]:
# Run over everything
cerebro.run()

2016-06-01, Close, 0.03
2016-06-01, BUY CREATE, 0.03
2016-06-02, BUY EXECUTED, Price: 0.03, Cost: 997.50, Comm 2.49
2016-06-02, Close, 0.03
2016-06-03, Close, 0.02
2016-06-03, SELL CREATE, 0.02
2016-06-04, SELL EXECUTED, Price: 0.02, Cost: 997.50, Comm: 2.32
2016-06-04, OPERATION PROFIT, GROSS -69.74, NET -74.55
2016-06-04, Close, 0.02
2016-06-05, Close, 0.02
2016-06-06, Close, 0.02
2016-06-07, Close, 0.03
2016-06-07, BUY CREATE, 0.03
2016-06-08, BUY EXECUTED, Price: 0.03, Cost: 923.14, Comm 2.31
2016-06-08, Close, 0.03
2016-06-09, Close, 0.03
2016-06-10, Close, 0.02
2016-06-10, SELL CREATE, 0.02
2016-06-11, SELL EXECUTED, Price: 0.02, Cost: 923.14, Comm: 2.23
2016-06-11, OPERATION PROFIT, GROSS -30.81, NET -35.35
2016-06-11, Close, 0.02
2016-06-12, Close, 0.02
2016-06-13, Close, 0.03
2016-06-13, BUY CREATE, 0.03
2016-06-14, BUY EXECUTED, Price: 0.03, Cost: 887.88, Comm 2.22
2016-06-14, Close, 0.03
2016-06-15, Close, 0.03
2016-06-16, Close, 0.03
2016-06-17, Close, 0.02
2016-06-17, SELL

2018-02-25, Close, 0.09
2018-02-26, Close, 0.08
2018-02-27, Close, 0.08
2018-02-27, SELL CREATE, 0.08
2018-02-28, SELL EXECUTED, Price: 0.08, Cost: 8647.90, Comm: 21.00
2018-02-28, OPERATION PROFIT, GROSS -247.23, NET -289.85
2018-02-28, Close, 0.08
2018-03-01, Close, 0.08
2018-03-02, Close, 0.08
2018-03-03, Close, 0.08
2018-03-04, Close, 0.08
2018-03-05, Close, 0.07
2018-03-06, Close, 0.08
2018-03-07, Close, 0.08
2018-03-08, Close, 0.07
2018-03-09, Close, 0.08
2018-03-09, BUY CREATE, 0.08
2018-03-10, BUY EXECUTED, Price: 0.08, Cost: 8358.78, Comm 20.90
2018-03-10, Close, 0.08
2018-03-11, Close, 0.08
2018-03-11, SELL CREATE, 0.08
2018-03-12, SELL EXECUTED, Price: 0.08, Cost: 8358.78, Comm: 20.48
2018-03-12, OPERATION PROFIT, GROSS -166.87, NET -208.25
2018-03-12, Close, 0.08
2018-03-12, BUY CREATE, 0.08
2018-03-13, BUY EXECUTED, Price: 0.08, Cost: 8151.05, Comm 20.38
2018-03-13, Close, 0.08
2018-03-13, SELL CREATE, 0.08
2018-03-14, SELL EXECUTED, Price: 0.08, Cost: 8151.05, Comm: 20.07

In [431]:
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Final Portfolio Value: 11138.26


In [360]:
cerebro.plot(iplot=False)

[[<Figure size 3360x2000 with 5 Axes>]]